<a href="https://colab.research.google.com/github/sakshikhandoba/LLM_Assessment/blob/main/LLM_Engineering_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Can algorithms like SMOTE/ROSE which synthesize new examples from the training data be called Generative AI?



SMOTE (Synthetic Minority Over-sampling Technique) and ROSE (Random Over-Sampling Examples) are data augmentation techniques specifically designed to address class imbalance in datasets by generating new synthetic samples. SMOTE generates new instances by interpolating between existing minority class examples, effectively creating new data points along the lines joining these instances in the feature space. ROSE, on the other hand, generates synthetic data by adding random noise to existing examples, producing variations of the original data points. Both techniques focus on enhancing the training dataset for classification tasks and rely on relatively straightforward algorithms. In contrast, generative AI, learn the underlying distribution of the training data and generate entirely new data instances that are not direct variations of existing data. These models involve adversarial training or probabilistic sampling from latent spaces, enabling them to create high-dimensional, high-quality data across various domains such as images, text, and audio.

#WikipediaQueryRun Tool

In [2]:
!pip install --upgrade --quiet  wikipedia

In [1]:
!pip install langchain==0.1.14
!pip install langchain-community==0.0.31
!pip install langchain_openai==0.1.1
!pip install wikipedia
!pip install html5lib
!pip install requests
!pip install chromadb==0.4.3
!pip install pysqlite3-binary
!pip install opentelemetry-exporter-otlp

In [3]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
#Initialize the tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

#Run the tool
wikipedia.run("Generative Artificial Intelligence")

"Page: Generative artificial intelligence\nSummary: Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora. Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.\nGenerative AI has uses across a wide range of industries, including sof

#Youtube Search Tool

In [5]:
#Install libraries
!pip install --upgrade --quiet  youtube_search
!pip install --upgrade --quiet  youtube-transcript-api

In [6]:
#Import
from langchain_community.tools import YouTubeSearchTool
from langchain_community.document_loaders import YoutubeLoader

In [7]:
#Initialize the tool
tool = YouTubeSearchTool()
video = tool.run("LangChain",1)

['https://www.youtube.com/watch?v=aywZrzNaKjs&pp=ygUJTGFuZ0NoYWlu', 'https://www.youtube.com/watch?v=1bUy-1hGZpI&pp=ygUJTGFuZ0NoYWlu']

In [8]:
import re
urls = re.findall(r'https?://\S+?(?=\')', video)

In [9]:
#Obtain the transcript
loader = YoutubeLoader.from_youtube_url(
    urls[0], add_video_info=False
)
loader.load()

[Document(page_content="blank chain what is it why should you use it and how does it work let's have a look Lang chain is an open source framework that allows developers working with AI to combine large language models like gbt4 with external sources of computation and data the framework is currently offered as a python or a JavaScript package typescript to be specific in this video we're going to start unpacking the python framework and we're going to see why the popularity of the framework is exploding right now especially after the introduction of gpt4 in March 2023 to understand what need Lang chain fills let's have a look at a practical example so by now we all know that chat typically or tpt4 has an impressive general knowledge we can ask it about almost anything and we'll get a pretty good answer suppose you want to know something specifically from your own data your own document it could be a book a PDF file a database with proprietary information link chain allows you to conne

#Semantic Similarity Selector

In [10]:
!pip install langchain_chroma
!pip install langchain_core

In [11]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [12]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model="ADA_RAG_DONO_DEMO",
    api_key="534036f31d14400b9f07a2cd7680af25",
    api_version="2024-02-01",
    azure_endpoint="https://dono-rag-demo-resource-instance.openai.azure.com"
)

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    # Number of examples to produce.
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [14]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [15]:
# Input is a measurement, so should select the tall/short example
print(similar_prompt.format(adjective="large"))

Give the antonym of every input

Input: tall
Output: short

Input: large
Output:


In [16]:
# You can add new examples to the SemanticSimilarityExampleSelector as well
similar_prompt.example_selector.add_example(
    {"input": "enthusiastic", "output": "apathetic"}
)
print(similar_prompt.format(adjective="passionate"))

Give the antonym of every input

Input: enthusiastic
Output: apathetic

Input: passionate
Output:


#N-Gram Overlap Selector

In [17]:
from langchain_community.example_selectors.ngram_overlap import (
    NGramOverlapExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a fictional translation task.
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

In [18]:
!pip install nltk

In [20]:
example_selector = NGramOverlapExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The threshold, at which selector stops.
    # It is set to -1.0 by default.
    threshold=-1.0,

)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the Spanish translation of every input",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

In [21]:
# An example input with large ngram overlap with "Spot can run."
# and no overlap with "My dog barks."
print(dynamic_prompt.format(sentence="Spot can run fast."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:


In [22]:
# You can add examples to NGramOverlapExampleSelector as well.
new_example = {"input": "Spot plays fetch.", "output": "Spot juega a buscar."}

example_selector.add_example(new_example)
print(dynamic_prompt.format(sentence="Spot can run fast."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:


In [23]:
# Setting small nonzero threshold
example_selector.threshold = 0.09
print(dynamic_prompt.format(sentence="Spot can play fetch."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: Spot can play fetch.
Output:


#XML Parser

In [24]:
from langchain.output_parsers import XMLOutputParser
from langchain_community.chat_models import ChatAnthropic
from langchain_core.prompts import PromptTemplate

In [25]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
    temperature=0,
    api_key="534036f31d14400b9f07a2cd7680af25",
    api_version="2024-02-01",
    azure_endpoint="https://dono-rag-demo-resource-instance.openai.azure.com",
    model="GPT_35_TURBO_DEMO_RAG_DEPLOYMENT_DONO"
)

In [26]:
actor_query = "Generate the shortened filmography for Tom Hanks."
output = model.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)
print(output.content)

<movie>Big</movie>
<movie>Splash</movie>
<movie>Forrest Gump</movie>
<movie>Apollo 13</movie>
<movie>Saving Private Ryan</movie>
<movie>Cast Away</movie>
<movie>The Green Mile</movie>
<movie>Toy Story (voice of Woody)</movie>
<movie>The Da Vinci Code</movie>
<movie>Bridge of Spies</movie>


In [27]:
!pip install defusedxml

In [28]:
parser = XMLOutputParser()

prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

output = chain.invoke({"query": actor_query})
print(output)

{'filmography': [{'film': [{'title': 'Splash'}, {'year': '1984'}]}, {'film': [{'title': 'Big'}, {'year': '1988'}]}, {'film': [{'title': 'Forrest Gump'}, {'year': '1994'}]}, {'film': [{'title': 'Apollo 13'}, {'year': '1995'}]}, {'film': [{'title': 'Toy Story'}, {'year': '1995'}]}, {'film': [{'title': 'Saving Private Ryan'}, {'year': '1998'}]}, {'film': [{'title': 'The Green Mile'}, {'year': '1999'}]}, {'film': [{'title': 'Cast Away'}, {'year': '2000'}]}, {'film': [{'title': 'The Da Vinci Code'}, {'year': '2006'}]}, {'film': [{'title': 'Captain Phillips'}, {'year': '2013'}]}]}


In [29]:
parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = prompt | model | parser

output = chain.invoke({"query": actor_query})

print(output)

{'movies': [{'actor': [{'name': 'Tom Hanks'}, {'film': [{'name': 'Forrest Gump'}, {'genre': 'Drama'}]}, {'film': [{'name': 'Saving Private Ryan'}, {'genre': 'War'}]}, {'film': [{'name': 'Cast Away'}, {'genre': 'Drama'}]}, {'film': [{'name': 'Toy Story'}, {'genre': 'Animation'}]}]}]}


In [30]:
for s in chain.stream({"query": actor_query}):
    print(s)

{'movies': [{'actor': [{'film': [{'name': 'Big'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'Comedy'}]}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Forrest Gump'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'Drama'}]}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Cast Away'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'Drama'}]}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Saving Private Ryan'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'War'}]}]}]}


#YAML Parser

In [31]:
from typing import List

from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI

In [32]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [33]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't skeletons fight each other?", punchline="They don't have the guts.")